This program uses LSTM for Time Series prognosis. 
The code uses Beijing Air Polution dataset which reports the level of air polution at the US embassy in Beijing, China for 5 years. 
The dataset can be downloaded from the UCI repository:
     https://archive.ics.uci.edu/ml/datasets/Beijing+PM2.5+Data
        


The original code can be found at https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/

    http://inmachineswetrust.com/posts/deep-learning-setup/

In [7]:
from pandas import read_csv
from datetime import datetime
def parse(x):
    return datetime.strptime(x, '%Y %m %d %H')
dataset = read_csv('dataset.csv',  parse_dates = [['year', 'month', 'day', 'hour']], index_col=0, date_parser=parse)


In [9]:
dataset.head(5)

,No,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
year_month_day_hour,,,,,,,,,
2010-01-01 00:00:00,1,NaN,-21,-11.0,1021.0,NW,1.79,0,0
2010-01-01 01:00:00,2,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2010-01-01 02:00:00,3,NaN,-21,-11.0,1019.0,NW,6.71,0,0
2010-01-01 03:00:00,4,NaN,-21,-14.0,1019.0,NW,9.84,0,0
2010-01-01 04:00:00,5,NaN,-20,-12.0,1018.0,NW,12.97,0,0


In [ ]:
dataset.drop('No', axis=1, inplace=True)
# manually specify column names
dataset.columns = ['pollution', 'dew', 'temp', 'press', 'wnd_dir', 'wnd_spd', 'snow', 'rain']
dataset.index.name = 'date'
# mark all NA values with 0
dataset['pollution'].fillna(0, inplace=True)
# drop the first 24 hours
dataset = dataset[24:]
# summarize first 5 rows
print(dataset.head(5))
# save to file
dataset.to_csv('pollution.csv')